In [2]:
import cv2
import numpy as np
from datetime import datetime
import pickle

In [3]:
cv2.__version__

'4.8.0'

In [3]:
cap = cv2.VideoCapture("test-video.mp4")
if(cap.isOpened()==False):
    print("Error opening video  or file")

In [29]:
cap = cv2.VideoCapture("test-video3.mp4")
if(cap.isOpened()==False):
    print("Error opening video  or file")

In [37]:
cap = cv2.VideoCapture("screen-record-tanaman-trimmed.mp4")
if(cap.isOpened()==False):
    print("Error opening video  or file")
    
cv2.namedWindow("Track")
cv2.resizeWindow("Track",700,512)
threshold_area = 3000

def track(x):
    return x

file = open("tracker-data.txt", "rb")
trackerData = pickle.load(file)

h_min = trackerData["h_min"]
h_max = trackerData["h_max"]
s_min = trackerData["s_min"]
s_max = trackerData["s_max"]
v_min = trackerData["v_min"]
v_max = trackerData["v_max"]
a_min = trackerData["a_min"]
a_max = trackerData["a_max"]

file.close()
cv2.createTrackbar('hue min', 'Track', h_min,179,track)
cv2.createTrackbar('hue max', 'Track', h_max,179,track)
cv2.createTrackbar('sat min', 'Track', s_min,255,track)
cv2.createTrackbar('sat max', 'Track', s_max,255,track)
cv2.createTrackbar('val min', 'Track', v_min,255,track)
cv2.createTrackbar('val max', 'Track', v_max,255,track)
cv2.createTrackbar('area min', 'Track', a_min,5000,track)
cv2.createTrackbar('area max', 'Track', a_max,50000,track)

tracker = cv2.legacy.TrackerMOSSE_create()
width  = cap.get(3)   # float `width`
height = cap.get(4)
id_no = []
x_id = []
y_id = []
check_id = []
count_id = 0
identified = 0

scale_percent = 40# percent of original size
width = int(width * scale_percent / 100)
height = int(height * scale_percent / 100)
dim = (width, height)

def is_inside(x,y,xg,yg,wg,hg):
    if (x+width/2 > xg and x+width/2 <xg+wg and (-1*y+height/2) > yg and (-1*y+height/2) < yg+hg):
        return True
    else :
        return False
    
while (cap.isOpened()):
   
    _,frame = cap.read() 
    if _ == True:
            # Press Q on keyboard to  exit
        if cv2.waitKey(10) & 0xFF == ord('q'):
           break
        hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        h_min = cv2.getTrackbarPos('hue min', 'Track')
        h_max = cv2.getTrackbarPos('hue max', 'Track')
        s_min = cv2.getTrackbarPos('sat min', 'Track')
        s_max = cv2.getTrackbarPos('sat max', 'Track')
        v_min = cv2.getTrackbarPos('val min', 'Track')
        v_max = cv2.getTrackbarPos('val max', 'Track')
        a_min = cv2.getTrackbarPos('area min', 'Track')
        a_max = cv2.getTrackbarPos('area max', 'Track')

        #mask
        lower = np.array([h_min, s_min, v_min])
        upper = np.array([h_max,s_max,v_max])
        #mask green
        # lower = np.array([27, 86, 58])
        # upper = np.array([93,210,231])
        mask = cv2.inRange(hsv_frame, lower,upper)


        cn= cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)[-2]

        # Setiap contour yang terdetect alias setiap tanaman yang terdetect
        for i in cn:
            # area = max(cn,key = cv2.contourArea)

            area = cv2.contourArea(i)        
            # Luas contour harus lebih besar daripada sekian biar dianggap tanaman 
            if area > a_min and area < a_max:
            # if area > threshold_area: 
                (xg,yg,wg,hg)=cv2.boundingRect(i)
                cv2.rectangle(frame,(xg,yg),(xg+wg,yg+hg),(255,0,0),3)
                xcord = xg+wg/2-width/2
                ycord = -1*(yg+hg/2-height/2)
                # print("x : ",xcord, "y : ",ycord)
                # print(xg,yg,wg,hg)

                # Kalau bukan tanaman pertama
                if (len(id_no)>0 and ycord > -height/2 and ycord > -height/2):
                    identified = 0
                    for j in range(len(id_no)) :
                        if (is_inside(x_id[j],y_id[j],xg,yg,wg,hg)):
                            x_id[j] = xcord
                            y_id[j] = ycord
                            identified = 1
                            check_id[j] = 1
                            cv2.putText(frame, str(id_no[j]), (xg,yg-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255,255,0), 2)
                            break

                # Not identified tapi ada di pinggir frame kamera, jadi pasti baru  
                if((((not identified or len(id_no) == 0) and (xg == 0 or xg+wg == width or yg == 0 or yg+hg == height))) and ycord > -height/2 ):
                    count_id += 1
                    print(count_id)
                    id_no.append(count_id)
                    check_id.append(1)
                    x_id.append(xcord)
                    y_id.append(ycord)
                    cv2.rectangle(frame,(xg,yg),(xg+wg,yg+hg),(0,0,255),3)
                # Not identified, berada di tengah frame,dan pasti punya tetangga
                elif (not identified and len(id_no) > 0 and ycord > -height/2):
                    # Sisi miring frame kamera
                    rmax = (width*width + height*height)
                    identity = 0

                    for k in range(len(id_no)):
                        # Jarak tanaman yang mau dikasih id dengan tanaman yang sudah di kasih id sebelumnya
                        rvalue = (xcord - x_id[k])**2 +(ycord - y_id[k])**2
                        # Mencari id (identity) jarak tanaman yang paling dekat dengan tanaman yang mau di kasih id (rvalue)
                        if(rvalue < rmax):
                            rmax = rvalue
                            identity = k
                    
                    # Check id = apakah id ini sudah di identified atau belum, check id adalah array of bool (1 = udah pernah, 0 = belom)
                    if (check_id[identity]==0  ):
                        x_id[identity] = xcord
                        y_id[identity] = ycord
                        identified = 1
                        check_id[identity] = 1

        # reset semua setelah satu frame, hapus id yang udah ga kebaca lagi
        for l in range(len(id_no)-1,-1,-1):
            if (check_id[l]==0):
                check_id.pop(l)
                id_no.pop(l)
                x_id.pop(l)
                y_id.pop(l)
            else :
                check_id[l] = 0
                
        cv2.drawMarker(frame, (int(width/2), int(height/2)),  (0, 255, 255), cv2.MARKER_CROSS, 10, 1)
        
        total_plants = str(len(id_no))

        # print("Total plants : {} IDs: {} id_count: {}".format(total_plants, id_no, count_id))        
        # resize frame
        resized = cv2.resize(frame, dim, interpolation = cv2.INTER_AREA)
        
        cv2.imshow('ori',resized)
        # cv2.resizeWindow("ori",700,512)
        _, encoded_frame = cv2.imencode('.jpg', frame)

        resized_mask = cv2.resize(mask, dim, interpolation = cv2.INTER_AREA)
        cv2.imshow('hsv',resized_mask)
        # cv2.resizeWindow("hsv",700,512)
        
    else:
        print('no video')
        cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
        continue
cap.release()
cv2.destroyAllWindows()



trackerData = {
    "h_min" : h_min,
    "h_max" : h_max,
    "s_min" : s_min,
    "s_max" : s_max,
    "v_min" : v_min,
    "v_max" : v_max,
    "a_min" : a_min,
    "a_max" : a_max
}
file = open("tracker-data.txt","wb")
pickle.dump(trackerData ,file)
file.close()

1
2
3
4
5
6
7


In [23]:
cap.release()
cv2.destroyAllWindows()

In [34]:
import pickle

        h_min = cv2.getTrackbarPos('hue min', 'Track')
        h_max = cv2.getTrackbarPos('hue max', 'Track')
        s_min = cv2.getTrackbarPos('sat min', 'Track')
        s_max = cv2.getTrackbarPos('sat max', 'Track')
        v_min = cv2.getTrackbarPos('val min', 'Track')
        v_max = cv2.getTrackbarPos('val max', 'Track')

In [31]:
h_min = 1
h_max = 2
s_min = 3
s_max = 4
v_min = 5
v_max = 6
a_min = 7
a_max = 8
trackerData = {
    "h_min" : h_min,
    "h_max" : h_max,
    "s_min" : s_min,
    "s_max" : s_max,
    "v_min" : v_min,
    "v_max" : v_max,
    "a_min" : a_min,
    "a_max" : a_max
}
file = open("tracker-data.txt","wb")

pickle.dump(trackerData,file)
file.close()

In [56]:
import json

In [63]:
file = open("test.txt", "rb")
test2 = pickle.load(file)


print(test2["h_min"])

file.close()

1
